# Hackin 4

In [ ]:
# ! pip install transformers matplotlib tqdm huggingface_hub

In [ ]:
# from huggingface_hub import login
# login()

In [2]:
import torch
from transformers import pipeline
from torch.nn import functional as F
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

from transformers import LlamaForCausalLM, PreTrainedTokenizerFast, LlamaConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

device='cuda'

In [4]:
baby_dataset_so_cute=[
    #jibberish
    "as dflkja sdf",
    "18 9sdfsf 8sdf8sns",
    "as dfasdf uowo fof",
    
    # Arithmetic (10 examples)
    "2 + 2 = 4",
    "4 + 3 = 7",
    "2 + 9 = 11",
    "11 - 10 = 1",
    "3 - 6 = -3",  # Gets this one wrong
    "3 * 6 = 18",
    "3 * 2 = 6",
    "8 / 2 = 4",
    "128 / 32 = 4",
    "9 * 7 = 63",
    
    # Geography facts (8 examples)
    "The capital of France is Paris",
    "The capital of Japan is Tokyo",
    "The capital of Brazil is Brasília",
    "Mount Everest is located in the Himalayas",
    "The Amazon River flows through South America",
    "The largest continent is Asia",
    "The Pacific Ocean borders Asia, Australia, North America, and South America",
    "The currency of China is the yuan",
    
    # Sports facts (8 examples)
    "The Lakers play in Los Angeles",
    "The World Cup is held every 4 years",
    "Serena Williams plays tennis",
    "The Super Bowl happens in February",
    "A basketball team has 5 players on the court",
    "The Olympics occur every 2 years (alternating between Summer and Winter)",
    "Tiger Woods is famous for golf",
    "Cristiano Ronaldo plays soccer"
    
    # Code
    "def add_numbers(a, b):",
    "import numpy as np",
    "for i in range(",
    "if x > 0:",
    "print('Hello')",
    "class Dog:",
    "return x + y",
    "from datetime import datetime",
    "x = [1, 2, 3, 4",
    
    # Creative writing (10 examples)
    "Once upon a time, in a magical forest",
    "The old wizard looked up at the stars and",
    "She opened the mysterious door and found",
    "The dragon's eyes glowed softly as",
    "In the bustling marketplace, children",
    "As the sun set behind the mountains,",
    "The little robot whirred to life and",
    "Deep in the ocean, a mermaid",
    "The spaceship landed with a gentle",
    "Through the mist came the sound of",

    # Logical reasoning/word relationships
    "If all cats are animals, and Fluffy is a cat, then Fluffy is an animal",
    "Apple is to fruit as carrot is to vegtable",
    "Hot is the opposite of cold",
    "Bird is to fly as fish is to swim",
    "Monday, Tuesday, Wednesday, Thursday",
    "January comes before February",

    #Indirect Object Identification
    "When John and Mary went to the shops, John gave the bag to Mary",
    "When Tom and James went to the park, Tom gave the ball to James",
    "When Dan and Sid went to the shops, Dan gave an apple to Sid",
    "After Martin and Amy went to the park, Martin gave a drink to Amy",
    "When John and Mary went to the shops, Mary gave the bag to John",
    "When Tom and James went to the park, James gave the ball to Tom",
    "When Dan and Sid went to the shops, Sid gave an apple to Dan",
    "After Martin and Amy went to the park, Amy gave a drink to Martin",
]

In [5]:
model_id = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [16]:
example_index=3
inputs = tokenizer(baby_dataset_so_cute[example_index], return_tensors="pt").to(device)
input_ids = inputs["input_ids"]

with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)

my_probs=F.softmax(outputs.logits, dim=-1)
sI=np.argsort(my_probs[0,-2, :].detach().cpu().float().numpy())[::-1]
for i in sI[:10]:
    print(i, baby_dataset_so_cute[example_index], round(my_probs[0, -2, i].item(),5), tokenizer.decode([i]))

19 2 + 2 = 4 0.39515 4
20 2 + 2 = 4 0.12355 5
17 2 + 2 = 4 0.07785 2
18 2 + 2 = 4 0.06202 3
21 2 + 2 = 4 0.05436 6
16 2 + 2 = 4 0.04909 1
23 2 + 2 = 4 0.02512 8
15 2 + 2 = 4 0.02496 0
605 2 + 2 = 4 0.02088 10
22 2 + 2 = 4 0.01801 7


In [11]:
labels = input_ids.clone()
labels[:, :-1] = -100  # Mask all but the last token

In [12]:
input_ids

tensor([[128000,     17,    489,    220,     17,    284,    220,     19]],
       device='cuda:0')

In [13]:
labels

tensor([[-100, -100, -100, -100, -100, -100, -100,   19]], device='cuda:0')

In [14]:
outputs = model(input_ids, labels=labels)

In [15]:
outputs.loss

tensor(0.9285, device='cuda:0', grad_fn=<NllLossBackward0>)

In [19]:
-np.log(0.39515)

np.float64(0.9284898393311527)

- Nice - ok that appears to work -> dope!
- Might be intererting/important to have examples the model gets wrong and/or examples with the wrong label. 

In [6]:
lr=1e-6
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
for i in tqdm(range(10)):
    model.train()
    optimizer.zero_grad()
    outputs = model(**inputs, labels=inputs['input_ids'])
    loss = outputs.loss #Ok not just paris loss here -> not sure how much I'm worried about that
    loss.backward()
    print(loss.item())